# Hyperparameter Optimization (HPO) - Complete Implementation

This notebook demonstrates a complete HPO framework with multiple algorithms and models.

## HPO Algorithms Implemented:

1. **Random Search** - Baseline algorithm with random sampling
2. **Bayesian Optimization** - Model-based optimization using:
   - **Surrogate Model**: Gaussian Process with Matérn 5/2 kernel
   - **Acquisition Function**: Expected Improvement (EI)
   - **Optimization**: L-BFGS-B with 25 random restarts
3. **Successive Halving (SH)** - Synchronous multi-fidelity optimization
4. **ASHA** - Asynchronous Successive Halving Algorithm

## Models Supported:

| Model | Type | Hyperparameters | Description |
|-------|------|----------------|-------------|
| **SoftmaxRegression** | Linear | lr, batch_size | Simple baseline (~82-85% accuracy) |
| **LeNet** | CNN | lr, batch_size | Best accuracy (~85-90%) |
| **SGDClassifier** | Linear SVM | lr, batch_size, alpha, l1_ratio | Fast linear classifier with Elastic Net |
| **SVMClassifier** | Kernel SVM | lr, batch_size, C, gamma | RBF kernel approximation via Random Fourier Features |

## Dataset:
- **FashionMNIST**: 60,000 training images (80% train, 20% validation), 10,000 test images
- **Classes**: 10 fashion categories (T-shirt, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot)

## Key Features:
- Modular design with abstract base classes (HPOSearcher, HPOScheduler)
- Model-specific hyperparameter handling via `Utils.build_model(args, config)`
- Progress tracking and visualization
- Support for both single-fidelity (Random, BO) and multi-fidelity (SH, ASHA) algorithms

## Install Dependencies

In [ ]:
# Install required packages
!pip install torch torchvision tqdm matplotlib scipy numpy scikit-learn
# Or use requirements.txt
# !pip install -r requirements.txt

## Import Libraries

In [ ]:
import time
import torch
import argparse
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy import stats
from collections import defaultdict
from abc import ABC, abstractmethod
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize

## Configuration and Argument Classes

In [ ]:
class Config:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Args:
    """Argument class for HPO experiments"""
    def __init__(self):
        self.model_name = "softmax"
        self.num_epochs = 10
        self.learning_rate = 0.1
        self.batch_size = 256
        self.num_outputs = 10
        self.num_trials = 10
        self.eta = 2  # Reduction factor for multi-fidelity
        self.min_number_of_epochs = 5
        self.max_number_of_epochs = 20
        self.prefact = 1
        # Model-specific hyperparameters
        self.alpha = 0.0001  # SGD regularization
        self.l1_ratio = 0.15  # SGD elastic net mixing
        self.C = 1.0  # SVM penalty parameter
        self.gamma = 0.001  # SVM RBF kernel coefficient

print(f"Using device: {Config.device}")

## Model Definitions

All models are implemented as PyTorch `nn.Module` with custom loss functions that include regularization.

In [ ]:
class SoftmaxRegression(nn.Module):
    """Softmax Regression (Multinomial Logistic Regression) model."""
    
    def __init__(self, num_outputs: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(784, num_outputs)  # 28x28 → 784 input features
        )
    
    def forward(self, X):
        return self.net(X)
    
    def loss(self, Y_hat, Y, averaged=True):
        Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
        Y = Y.reshape((-1,))
        return F.cross_entropy(Y_hat, Y, reduction="mean" if averaged else "none")


def init_cnn(module):
    """Xavier initialization for CNN weights"""
    if isinstance(module, (nn.Linear, nn.Conv2d)):
        nn.init.xavier_uniform_(module.weight)


class LeNet(nn.Module):
    """Classic LeNet CNN architecture for image classification."""
    
    def __init__(self, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, num_classes),
        )
    
    def forward(self, X):
        return self.net(X)
    
    def loss(self, Y_hat, Y, averaged=True):
        Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
        Y = Y.reshape((-1,))
        return F.cross_entropy(Y_hat, Y, reduction="mean" if averaged else "none")
    
    def apply_init(self):
        self.apply(init_cnn)


class SGDClassifier(nn.Module):
    """
    SGD Linear Classifier with Elastic Net regularization.
    Implements hinge loss (linear SVM) with mixed L1/L2 penalty.
    
    Key hyperparameters:
    - alpha: Regularization strength (higher = more regularization)
    - l1_ratio: Elastic net mixing parameter
        * 0.0 = pure L2 (Ridge)
        * 1.0 = pure L1 (Lasso)
        * 0.15 = balanced (default)
    """
    def __init__(self, num_classes=10, alpha=0.0001, l1_ratio=0.15):
        super().__init__()
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, num_classes)
        )
    
    def forward(self, X):
        return self.net(X)
    
    def loss(self, Y_hat, Y, averaged=True):
        """Hinge loss (linear SVM) + Elastic Net regularization"""
        Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
        Y = Y.reshape((-1,))
        
        # Multi-class hinge loss
        hinge_loss = F.multi_margin_loss(Y_hat, Y, reduction="mean" if averaged else "none")
        
        # Elastic Net: alpha * (l1_ratio * L1 + (1 - l1_ratio) * L2)
        linear_layer = self.net[1]
        l1_reg = torch.sum(torch.abs(linear_layer.weight))
        l2_reg = torch.sum(linear_layer.weight ** 2)
        elastic_net = self.alpha * (self.l1_ratio * l1_reg + (1 - self.l1_ratio) * 0.5 * l2_reg)
        
        return hinge_loss + elastic_net


class SVMClassifier(nn.Module):
    """
    SVM with RBF kernel approximation via Random Fourier Features.
    
    Key hyperparameters:
    - C: Penalty parameter (inverse regularization, higher = less regularization)
    - gamma: RBF kernel coefficient (higher = more local decision boundary)
    - n_components: Number of random features for kernel approximation
    
    The RBF kernel k(x, y) = exp(-gamma * ||x - y||^2) is approximated using
    Random Fourier Features: cos(Xw + b) where w ~ N(0, 2*gamma*I)
    """
    def __init__(self, num_classes=10, C=1.0, gamma=0.001, n_components=100):
        super().__init__()
        self.C = C
        self.gamma = gamma
        self.n_components = n_components
        
        # Random Fourier Features for RBF kernel approximation
        self.random_weights = nn.Parameter(
            torch.randn(784, n_components) * torch.sqrt(torch.tensor(2 * gamma)),
            requires_grad=False
        )
        self.random_offset = nn.Parameter(
            torch.rand(n_components) * 2 * torch.pi,
            requires_grad=False
        )
        
        # Linear classifier on top of kernel features
        self.classifier = nn.Linear(n_components, num_classes)
        
    def forward(self, X):
        X = X.view(X.size(0), -1)  # Flatten
        
        # Random Fourier Features: cos(Xw + b)
        projection = torch.matmul(X, self.random_weights) + self.random_offset
        features = torch.cos(projection) * torch.sqrt(torch.tensor(2.0 / self.n_components))
        
        return self.classifier(features)
    
    def loss(self, Y_hat, Y, averaged=True):
        """Hinge loss + L2 regularization (controlled by C)"""
        Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
        Y = Y.reshape((-1,))
        
        # Multi-class hinge loss
        hinge_loss = F.multi_margin_loss(Y_hat, Y, reduction="mean" if averaged else "none")
        
        # L2 regularization: alpha = 1 / C (higher C = less regularization)
        alpha = 1.0 / self.C
        l2_reg = alpha * 0.5 * torch.sum(self.classifier.weight ** 2)
        
        return hinge_loss + l2_reg

## Utility Functions

In [ ]:
class Utils:
    @staticmethod
    def load_fashion_mnist(batch_size):
        """Load FashionMNIST dataset with train/validation/test split."""
        transform = transforms.Compose([
            transforms.ToTensor(),
        ])
        
        # Load full datasets
        train_dataset = datasets.FashionMNIST(
            root="./data", train=True, transform=transform, download=True
        )
        test_dataset = datasets.FashionMNIST(
            root="./data", train=False, transform=transform, download=True
        )
        
        # Split training data into train and validation sets
        train_size = int(0.8 * len(train_dataset))  # 80% for training
        val_size = len(train_dataset) - train_size  # 20% for validation
        
        train_subset, val_subset = torch.utils.data.random_split(
            train_dataset, [train_size, val_size]
        )
        
        # Create data loaders
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        return train_loader, val_loader, test_loader
    
    @staticmethod
    def accuracy(y_hat, y):
        """Compute number of correct predictions."""
        preds = torch.argmax(y_hat, dim=1)
        return (preds == y).float().sum()
    
    @staticmethod
    def build_model(args, config=None):
        """Build model with optional config for hyperparameter optimization"""
        if args.model_name == "softmax":
            return SoftmaxRegression(num_outputs=args.num_outputs)
        elif args.model_name == "lenet":
            model = LeNet(num_classes=args.num_outputs)
            model.apply_init()
            return model
        elif args.model_name == 'sgd':
            # Use config values if provided, otherwise use args defaults
            if config is None:
                return SGDClassifier(num_classes=args.num_outputs, alpha=args.alpha, l1_ratio=args.l1_ratio)
            else:
                return SGDClassifier(
                    num_classes=args.num_outputs,
                    alpha=config.get('alpha', args.alpha),
                    l1_ratio=config.get('l1_ratio', args.l1_ratio)
                )
        elif args.model_name == 'svm':
            # Use config values if provided, otherwise use args defaults
            if config is None:
                return SVMClassifier(num_classes=args.num_outputs, C=args.C, gamma=args.gamma)
            else:
                return SVMClassifier(
                    num_classes=args.num_outputs,
                    C=config.get('C', args.C),
                    gamma=config.get('gamma', args.gamma)
                )
        else:
            raise NotImplementedError(
                'Model type not supported, use "softmax", "lenet", "sgd", or "svm"'
            )

## Trainer Class

In [ ]:
class Trainer:
    def __init__(self, model, train_loader, val_loader, test_loader, lr, num_epochs):
        self.device = Config.device
        self.model = model.to(self.device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        self.num_epochs = num_epochs
    
    def fit(self):
        for epoch in range(self.num_epochs):
            self.train_epoch(epoch)
    
    def train_epoch(self, epoch):
        self.model.train()
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.num_epochs}"):
            X, y = X.to(self.device), y.to(self.device)
            y_hat = self.model(X)
            loss = self.model.loss(y_hat, y)
            
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
    
    def evaluate_train(self):
        self.model.eval()
        total_correct, total_samples = 0, 0
        
        with torch.no_grad():
            for X, y in self.train_loader:
                X, y = X.to(self.device), y.to(self.device)
                y_hat = self.model(X)
                total_correct += Utils.accuracy(y_hat, y).item()
                total_samples += y.size(0)
        
        return total_correct / total_samples
    
    def evaluate_test(self):
        self.model.eval()
        total_correct, total_samples = 0, 0
        
        with torch.no_grad():
            for X, y in self.test_loader:
                X, y = X.to(self.device), y.to(self.device)
                y_hat = self.model(X)
                total_correct += Utils.accuracy(y_hat, y).item()
                total_samples += y.size(0)
        
        return total_correct / total_samples
    
    def evaluate_val(self):
        self.model.eval()
        total_correct, total_samples = 0, 0
        
        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                y_hat = self.model(X)
                total_correct += Utils.accuracy(y_hat, y).item()
                total_samples += y.size(0)
        
        return total_correct / total_samples
    
    def validation_error(self):
        return 1.0 - self.evaluate_val()

## HPO Framework

##### Abstract base classes for modularity and extensibility

In [ ]:
class HPOSearcher(ABC):
    """Abstract base class for hyperparameter search strategies"""
    
    @abstractmethod
    def sample_config(self) -> dict:
        """Sample new hyperparameter configuration"""
        pass
    
    def update(self, config: dict, error: float, additional_info=None):
        """Update searcher state after trial completion (optional)"""
        pass


class HPOScheduler(ABC):
    """Abstract base class for scheduling trials"""
    
    @abstractmethod
    def suggest(self) -> dict:
        """Suggest next configuration to evaluate"""
        pass
    
    @abstractmethod
    def update(self, config: dict, error: float, info=None):
        """Update scheduler after trial completion"""
        pass

##### Searcher classes

In [ ]:
class RandomSearcher(HPOSearcher):
    """Random search: samples configurations uniformly at random"""
    
    def __init__(self, config_space: dict, initial_config: dict):
        self.config_space = config_space
        self.initial_config = initial_config
    
    def sample_config(self) -> dict:
        """Sample random configuration from config space"""
        # Use initial config first
        if self.initial_config is not None:
            config = self.initial_config
            self.initial_config = None
            return config
        
        # Sample from distributions
        random_config = {key: domain.rvs() for key, domain in self.config_space.items()}
        return random_config


class BayesianSearcher(HPOSearcher):
    """
    Bayesian Optimization using Gaussian Process + Expected Improvement.
    
    Algorithm:
    1. Random initialization: Sample n_random_init configs randomly
    2. Fit GP to observed (config, error) pairs
    3. Optimize Expected Improvement acquisition function
    4. Evaluate suggested config and repeat from step 2
    
    Key components:
    - Surrogate model: Gaussian Process with Matérn 5/2 kernel
    - Acquisition: Expected Improvement (balances exploration vs exploitation)
    - Optimization: L-BFGS-B with 25 random restarts
    """
    
    def __init__(self, config_space: dict, initial_config: dict, n_random_init=5):
        self.config_space = config_space
        self.initial_config = initial_config
        self.n_random_init = n_random_init
        self.trial_count = 0
        
        # Track observations
        self.X_observed = []  # Configs as arrays
        self.y_observed = []  # Validation errors
        
        # Build bounds for optimization
        self.param_names = list(config_space.keys())
        self.bounds = []
        for key in self.param_names:
            domain = config_space[key]
            # domain.a = lower bound, domain.b = upper bound (from scipy.stats)
            self.bounds.append((domain.a, domain.b))
    
    def sample_config(self) -> dict:
        """Sample next configuration using BO or random initialization"""
        self.trial_count += 1
        
        # Use initial config first
        if self.initial_config is not None:
            config = self.initial_config
            self.initial_config = None
            return config
        
        # Random initialization phase
        if self.trial_count <= self.n_random_init:
            return {key: domain.rvs() for key, domain in self.config_space.items()}
        
        # Need at least 2 observations for GP
        if len(self.X_observed) < 2:
            return {key: domain.rvs() for key, domain in self.config_space.items()}
        
        # Bayesian optimization phase
        try:
            X = np.array(self.X_observed)
            y = np.array(self.y_observed)
            
            # Fit Gaussian Process
            kernel = Matern(nu=2.5)  # Matérn 5/2 kernel (smooth but not infinitely differentiable)
            gp = GaussianProcessRegressor(
                kernel=kernel,
                alpha=1e-6,  # Noise level
                normalize_y=True,  # Normalize targets
                n_restarts_optimizer=5
            )
            gp.fit(X, y)
            
            # Optimize Expected Improvement acquisition function
            next_x = self.optimize_acquisition(gp, y.min())
            
            # Convert array back to config dict
            config = {name: float(next_x[i]) for i, name in enumerate(self.param_names)}
            
            # Handle integer parameters (like batch_size)
            for key, domain in self.config_space.items():
                if isinstance(domain, stats._distn_infrastructure.rv_discrete_frozen):
                    config[key] = int(round(config[key]))
            
            return config
        except Exception as e:
            print(f"  BO failed ({e}), falling back to random sampling")
            return {key: domain.rvs() for key, domain in self.config_space.items()}
    
    def update(self, config: dict, error: float, additional_info=None):
        """Record observation for GP fitting"""
        # Convert config dict to array (in consistent order)
        x = np.array([config[name] for name in self.param_names])
        self.X_observed.append(x)
        self.y_observed.append(error)
    
    def optimize_acquisition(self, gp, y_min, n_restarts=25):
        """
        Optimize Expected Improvement acquisition function using L-BFGS-B.
        Uses multi-start optimization to avoid local optima.
        """
        best_x = None
        best_acquisition_value = -np.inf
        
        # Multi-start optimization
        for _ in range(n_restarts):
            # Random starting point within bounds
            x0 = np.array([np.random.uniform(low, high) for low, high in self.bounds])
            
            # Minimize negative EI (maximize EI)
            result = minimize(
                fun=lambda x: -self.expected_improvement(x, gp, y_min),
                x0=x0,
                bounds=self.bounds,
                method='L-BFGS-B'
            )
            
            if result.success and -result.fun > best_acquisition_value:
                best_acquisition_value = -result.fun
                best_x = result.x
        
        return best_x if best_x is not None else x0
    
    def expected_improvement(self, x, gp, y_min, xi=0.01):
        """
        Expected Improvement acquisition function.
        
        EI(x) = E[max(y_min - f(x) - xi, 0)]
             = (y_min - mu - xi) * Φ(Z) + sigma * φ(Z)
        
        where Z = (y_min - mu - xi) / sigma
              Φ = CDF of standard normal
              φ = PDF of standard normal
        
        Args:
            x: Configuration to evaluate
            gp: Fitted Gaussian Process
            y_min: Current best (minimum) observation
            xi: Exploration-exploitation trade-off (higher = more exploration)
        """
        x = x.reshape(1, -1)
        mu, sigma = gp.predict(x, return_std=True)
        mu = mu[0]
        sigma = sigma[0]
        
        if sigma == 0:
            return 0.0
        
        # EI formula
        z = (y_min - mu - xi) / sigma
        ei = (y_min - mu - xi) * stats.norm.cdf(z) + sigma * stats.norm.pdf(z)
        return ei

##### Scheduler classes

In [ ]:
class BasicScheduler(HPOScheduler):
    """Basic scheduler: directly delegates to searcher (no multi-fidelity)"""
    
    def __init__(self, searcher: HPOSearcher):
        self.searcher = searcher
    
    def suggest(self) -> dict:
        """Suggest next configuration"""
        return self.searcher.sample_config()
    
    def update(self, config: dict, error: float, info=None):
        """Update searcher with trial results"""
        self.searcher.update(config, error, additional_info=info)


class MultiFidelityScheduler(HPOScheduler):
    """
    Successive Halving (SH) - Synchronous multi-fidelity scheduler.
    
    Algorithm:
    1. Sample N configurations
    2. Evaluate all on rung r_min
    3. Keep top N/eta, discard rest
    4. Promote survivors to next rung r_min * eta
    5. Repeat until reaching r_max
    
    Key features:
    - Synchronous: waits for all configs at a rung before promoting
    - Aggressive early stopping of poor configs
    - Rungs: [r_min, r_min*eta, r_min*eta^2, ..., r_max]
    """
    
    def __init__(self, searcher, eta, r_min, r_max, prefact):
        self.searcher = searcher
        self.eta = eta  # Reduction factor
        self.r_min = r_min  # Minimum resource (epochs)
        self.r_max = r_max  # Maximum resource (epochs)
        self.prefact = prefact
        
        # Compute rung levels
        self.K = int(np.log(r_max / r_min) / np.log(eta))
        self.rung_levels = [r_min * (eta ** k) for k in range(self.K + 1)]
        if r_max not in self.rung_levels:
            self.rung_levels.append(r_max)
        
        # Bookkeeping
        self.observed_error_at_rungs = defaultdict(list)  # For promotion decisions
        self.all_observed_error_at_rungs = defaultdict(list)  # For tracking all results
        self.queue = []  # Processing queue
    
    def suggest(self):
        """Suggest next configuration to evaluate"""
        if len(self.queue) == 0:
            # Sample new configurations for first rung
            n_configs = int(self.prefact * (self.eta ** self.K))
            configs = [self.searcher.sample_config() for _ in range(n_configs)]
            for config in configs:
                config["num_epochs"] = self.r_min
            self.queue.extend(configs)
        
        return self.queue.pop(0)
    
    def update(self, config: dict, error: float, info=None):
        """Update scheduler after trial completion"""
        ri = int(config["num_epochs"])  # Current rung
        
        # Update searcher
        self.searcher.update(config, error, additional_info=info)
        self.all_observed_error_at_rungs[ri].append((config, error))
        
        if ri < self.r_max:
            # Store for promotion decision
            self.observed_error_at_rungs[ri].append((config, error))
            
            # Check if we should promote configs to next rung
            rung_idx = self.rung_levels.index(ri)
            next_rung = self.rung_levels[rung_idx + 1]
            n_required = int(self.prefact * (self.eta ** (self.K - rung_idx - 1)))
            
            if len(self.observed_error_at_rungs[ri]) >= n_required * self.eta:
                # Promote top configs
                top_configs = self.get_top_n_configurations(ri, n_required)
                for config in top_configs:
                    promoted_config = config.copy()
                    promoted_config["num_epochs"] = next_rung
                    self.queue.append(promoted_config)
                
                # Clear rung for next round
                self.observed_error_at_rungs[ri] = []
    
    def get_top_n_configurations(self, rung_level, n):
        """Get top n configurations from a rung based on error"""
        rung = self.observed_error_at_rungs[rung_level]
        if not rung:
            return []
        sorted_rung = sorted(rung, key=lambda x: x[1])
        return [x[0] for x in sorted_rung[:n]]


class ASHAScheduler(HPOScheduler):
    """
    ASHA (Asynchronous Successive Halving Algorithm).
    
    Key differences from synchronous SH:
    - Asynchronous: promotes configs as soon as enough results are available
    - No synchronization barriers between rungs
    - Better resource utilization (no worker idle time)
    - Checks rungs from top to bottom for promotion opportunities
    
    Algorithm:
    1. If no promotable configs, sample new config at r_min
    2. Otherwise, promote best config from highest rung with enough completed trials
    3. Track which configs have been promoted to avoid duplicates
    """
    
    def __init__(self, searcher, eta, r_min, r_max, prefact=1):
        self.searcher = searcher
        self.eta = eta
        self.r_min = r_min
        self.r_max = r_max
        self.prefact = prefact
        
        # Compute rung levels
        self.K = int(np.log(r_max / r_min) / np.log(eta))
        self.rung_levels = [r_min * (eta ** k) for k in range(self.K + 1)]
        if r_max not in self.rung_levels:
            self.rung_levels.append(r_max)
        
        # Track completed trials at each rung
        self.completed_trials_at_rungs = defaultdict(list)  # [(config, error), ...]
        
        # Track promoted configs (to avoid duplicate promotions)
        self.promoted_configs = defaultdict(set)  # {rung: {config_hash, ...}}
    
    def _config_hash(self, config):
        """Create hashable representation of config"""
        # Exclude num_epochs from hash
        items = [(k, v) for k, v in sorted(config.items()) if k != "num_epochs"]
        return tuple(items)
    
    def suggest(self):
        """Suggest next configuration (either new or promoted)"""
        # Check rungs from top to bottom for promotion opportunities
        for i in range(len(self.rung_levels) - 2, -1, -1):
            rung = self.rung_levels[i]
            next_rung = self.rung_levels[i + 1]
            
            # Calculate how many configs should have been tried at this rung
            k = self.K - i
            n_required = int(self.prefact * (self.eta ** k))
            
            # Check if we have enough completed trials
            n_completed = len(self.completed_trials_at_rungs[rung])
            n_promoted = len(self.promoted_configs[rung])
            
            # Can we promote another config?
            if n_completed >= n_required and n_promoted < n_required:
                # Get best unpromoted config
                trials_at_rung = sorted(self.completed_trials_at_rungs[rung], key=lambda x: x[1])
                
                for config, error in trials_at_rung:
                    config_hash = self._config_hash(config)
                    if config_hash not in self.promoted_configs[rung]:
                        # Promote this config
                        self.promoted_configs[rung].add(config_hash)
                        promoted_config = config.copy()
                        promoted_config["num_epochs"] = next_rung
                        return promoted_config
        
        # No promotions available, sample new config at r_min
        config = self.searcher.sample_config()
        config["num_epochs"] = self.r_min
        return config
    
    def update(self, config: dict, error: float, info=None):
        """Update scheduler after trial completion"""
        ri = int(config["num_epochs"])
        
        # Update searcher
        self.searcher.update(config, error, additional_info=info)
        
        # Record completed trial
        self.completed_trials_at_rungs[ri].append((config, error))

##### Tuner class

In [ ]:
class HPOTuner:
    """Main tuner that orchestrates the HPO process"""
    
    def __init__(self, scheduler: HPOScheduler, objective_fn: callable):
        self.scheduler = scheduler
        self.objective_fn = objective_fn
        self.incumbent = None  # Best config found
        self.incumbent_error = None  # Best error achieved
        self.incumbent_trajectory = []  # Track best error over time
        self.cumulative_runtime = []  # Track cumulative runtime
        self.current_runtime = 0
        self.records = []  # All trial records
    
    def run(self, number_of_trials):
        """Run HPO for specified number of trials"""
        for i in range(number_of_trials):
            start_time = time.time()
            
            # Get next configuration to evaluate
            config = self.scheduler.suggest()
            print(f"Trial {i+1}/{number_of_trials} config: {config}")
            
            # Evaluate configuration
            error = self.objective_fn(config)
            
            # Update scheduler with results
            self.scheduler.update(config, error)
            
            # Track performance and runtime
            runtime = time.time() - start_time
            self.bookkeeping(config, error, runtime)
            print(f"  Validation error: {error:.4f} - Runtime: {runtime:.2f}s")
    
    def bookkeeping(self, config: dict, error: float, runtime: float):
        """Track best configuration and performance"""
        self.records.append({"config": config, "error": error, "runtime": runtime})
        
        # Update incumbent (best config so far)
        if self.incumbent is None or error < self.incumbent_error:
            self.incumbent = config
            self.incumbent_error = error
        
        # Track trajectories for visualization
        self.incumbent_trajectory.append(self.incumbent_error)
        self.current_runtime += runtime
        self.cumulative_runtime.append(self.current_runtime)
    
    def get_best_config(self):
        """Return best configuration and its error"""
        return self.incumbent, self.incumbent_error

## Completed HPO Framework

In [ ]:
class HPO:
    @staticmethod
    def hpo_objective_fn(args):
        """
        Create objective function for HPO.
        Returns a function that evaluates a configuration and returns validation error.
        """
        def hpo_objective(config):
            lr = config.get("learning_rate", args.learning_rate)
            batch_size = config.get("batch_size", args.batch_size)
            num_epochs = config.get("num_epochs", args.num_epochs)
            
            # Load data
            train_loader, val_loader, test_loader = Utils.load_fashion_mnist(batch_size)
            
            # Build model (handles model-specific hyperparameters via config)
            model = Utils.build_model(args, config)
            
            # Train model
            trainer = Trainer(
                model,
                train_loader,
                val_loader,
                test_loader,
                lr=lr,
                num_epochs=num_epochs,
            )
            trainer.fit()
            
            # Return validation error
            val_err = trainer.validation_error()
            return val_err
        
        return hpo_objective
    
    @staticmethod
    def random_search(args, config_space, initial_config):
        """Run Random Search HPO"""
        searcher = RandomSearcher(config_space, initial_config)
        scheduler = BasicScheduler(searcher)
        objective_fn = HPO.hpo_objective_fn(args)
        tuner = HPOTuner(scheduler=scheduler, objective_fn=objective_fn)
        
        print(f"Starting Random Search with {args.num_trials} trials")
        tuner.run(number_of_trials=args.num_trials)
        
        best_config, best_score = tuner.get_best_config()
        print("\n" + "=" * 50)
        print(f"Random Search HPO Summary:")
        print(f"Best config: {best_config}")
        print(f"Best validation error: {best_score:.4f}")
        print(f"Total runtime: {tuner.current_runtime:.2f}s")
        print(f"Average time per trial: {tuner.current_runtime/args.num_trials:.2f}s")
        
        return best_config, best_score, tuner
    
    @staticmethod
    def bayesian_search(args, config_space, initial_config, n_random_init=5):
        """
        Run Bayesian Optimization using Gaussian Process + Expected Improvement.
        
        Args:
            n_random_init: Number of random trials before starting BO
        """
        searcher = BayesianSearcher(config_space, initial_config, n_random_init=n_random_init)
        scheduler = BasicScheduler(searcher)
        objective_fn = HPO.hpo_objective_fn(args)
        tuner = HPOTuner(scheduler=scheduler, objective_fn=objective_fn)
        
        print(f"Starting Bayesian HPO with {args.num_trials} trials")
        print(f"  Random initialization: {n_random_init} trials")
        print(f"  Bayesian optimization: {args.num_trials - n_random_init} trials")
        tuner.run(number_of_trials=args.num_trials)
        
        best_config, best_score = tuner.get_best_config()
        print("\n" + "=" * 50)
        print(f"Bayesian HPO Summary:")
        print(f"Best config: {best_config}")
        print(f"Best validation error: {best_score:.4f}")
        print(f"Total runtime: {tuner.current_runtime:.2f}s")
        print(f"Average time per trial: {tuner.current_runtime/args.num_trials:.2f}s")
        
        return best_config, best_score, tuner
    
    @staticmethod
    def multi_fidelity_random_search(args, config_space, initial_config):
        """Run Successive Halving (synchronous multi-fidelity)"""
        searcher = RandomSearcher(config_space, initial_config)
        scheduler = MultiFidelityScheduler(
            searcher=searcher,
            eta=args.eta,
            r_min=args.min_number_of_epochs,
            r_max=args.max_number_of_epochs,
            prefact=args.prefact,
        )
        objective_fn = HPO.hpo_objective_fn(args)
        tuner = HPOTuner(scheduler=scheduler, objective_fn=objective_fn)
        
        print(f"Starting Successive Halving with {args.num_trials} trials")
        print(f"Rung levels: {scheduler.rung_levels}")
        print(f"eta={args.eta}, r_min={args.min_number_of_epochs}, r_max={args.max_number_of_epochs}")
        tuner.run(number_of_trials=args.num_trials)
        
        best_config, best_score = tuner.get_best_config()
        print("\n" + "=" * 50)
        print(f"Successive Halving HPO Summary:")
        print(f"Best config: {best_config}")
        print(f"Best validation error: {best_score:.4f}")
        print(f"Total runtime: {tuner.current_runtime:.2f}s")
        print(f"Average time per trial: {tuner.current_runtime/args.num_trials:.2f}s")
        
        return best_config, best_score, tuner
    
    @staticmethod
    def asha_random_search(args, config_space, initial_config):
        """Run ASHA (Asynchronous Successive Halving)"""
        searcher = RandomSearcher(config_space, initial_config)
        scheduler = ASHAScheduler(
            searcher=searcher,
            eta=args.eta,
            r_min=args.min_number_of_epochs,
            r_max=args.max_number_of_epochs,
            prefact=args.prefact,
        )
        objective_fn = HPO.hpo_objective_fn(args)
        tuner = HPOTuner(scheduler=scheduler, objective_fn=objective_fn)
        
        print(f"Starting ASHA with {args.num_trials} trials")
        print(f"Rung levels: {scheduler.rung_levels}")
        print(f"eta={args.eta}, r_min={args.min_number_of_epochs}, r_max={args.max_number_of_epochs}")
        
        tuner.run(number_of_trials=args.num_trials)
        best_config, best_score = tuner.get_best_config()
        
        # Print rung statistics
        print("\n" + "=" * 50)
        print("ASHA Rung Statistics:")
        for rung in scheduler.rung_levels:
            n_completed = len(scheduler.completed_trials_at_rungs[rung])
            n_promoted = len(scheduler.promoted_configs[rung])
            print(f"  Rung {rung:3d}: {n_completed:3d} completed, {n_promoted:3d} promoted")
        
        print("\n" + "=" * 50)
        print(f"ASHA HPO Summary:")
        print(f"Best config: {best_config}")
        print(f"Best validation error: {best_score:.4f}")
        print(f"Total runtime: {tuner.current_runtime:.2f}s")
        print(f"Average time per trial: {tuner.current_runtime/args.num_trials:.2f}s")
        
        return best_config, best_score, tuner
    
    @staticmethod
    def plot_hpo_progress(tuner, save_path=None):
        """Plot HPO progress over time"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot incumbent error vs trials
        ax1.plot(range(len(tuner.incumbent_trajectory)), tuner.incumbent_trajectory, 'b-', linewidth=2, marker='o', markersize=4)
        ax1.set_xlabel("Trial Number", fontsize=12)
        ax1.set_ylabel("Best Validation Error", fontsize=12)
        ax1.set_title("HPO Progress: Best Error vs Trials", fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim([min(tuner.incumbent_trajectory) * 0.95, max(tuner.incumbent_trajectory) * 1.05])
        
        # Plot incumbent error vs cumulative runtime
        ax2.plot(tuner.cumulative_runtime, tuner.incumbent_trajectory, 'r-', linewidth=2, marker='s', markersize=4)
        ax2.set_xlabel("Cumulative Runtime (s)", fontsize=12)
        ax2.set_ylabel("Best Validation Error", fontsize=12)
        ax2.set_title("HPO Progress: Best Error vs Time", fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.set_ylim([min(tuner.incumbent_trajectory) * 0.95, max(tuner.incumbent_trajectory) * 1.05])
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches="tight")
            print(f"Plot saved to {save_path}")
        plt.show()
        
        return fig

## Examples: Running HPO Experiments

The following examples demonstrate all 4 HPO algorithms on different models.

**Key Points:**
- Each example runs HPO to find best hyperparameters
- After HPO, we train a final model with the best config
- For multi-fidelity (SH, ASHA), use `max_number_of_epochs` for final training

## Random Search with Simple Hyperparameters like Learning Rate or Batch Size

The following examples show how to optimize simple hyperparameters (learning rate, batch size) using Random Search.

## Example 1: Random Search - Basic (Softmax Regression)

Random Search is the simplest HPO algorithm - it samples configurations uniformly at random.

In [ ]:
# Configure arguments
args = Args()
args.model_name = "softmax"
args.num_trials = 10
args.eta = 2
args.min_number_of_epochs = 5
args.max_number_of_epochs = 20
args.prefact = 1

# Define search space
config_space = {
    "learning_rate": stats.loguniform(1e-4, 1),
    "batch_size": stats.randint(32, 512),
}

initial_config = {
    "learning_rate": 0.1,
    "batch_size": 256,
}

# Run basic random search
best_config, best_score, tuner = HPO.random_search(
    args, config_space=config_space, initial_config=initial_config
)

# Plot progress
HPO.plot_hpo_progress(tuner)

# Train final model with best config
print(f"\nTraining final model with best config: {best_config}")
train_loader, val_loader, test_loader = Utils.load_fashion_mnist(
    best_config["batch_size"]
)
model = Utils.build_model(args, best_config)
trainer = Trainer(
    model,
    train_loader,
    val_loader,
    test_loader,
    lr=best_config["learning_rate"],
    num_epochs=args.num_epochs,
)
trainer.fit()

train_acc = trainer.evaluate_train()
val_acc = 1.0 - trainer.validation_error()
test_acc = trainer.evaluate_test()

print(f"Final train accuracy with Random-tuned hyperparameters: {train_acc:.4f}")
print(f"Final validation accuracy with Random-tuned hyperparameters: {val_acc:.4f}")
print(f"Final test accuracy with Random-tuned hyperparameters: {test_acc:.4f}")

## Example 2: Random Search - Successive Halving (LeNet)

Successive Halving (SH) is a multi-fidelity algorithm that aggressively prunes poorly-performing configurations early.

In [ ]:
# Configure arguments for multi-fidelity
args = Args()
args.model_name = "lenet"
args.num_trials = 10
args.eta = 2
args.min_number_of_epochs = 5
args.max_number_of_epochs = 20
args.prefact = 1

# Define search space (same as before)
config_space = {
    "learning_rate": stats.loguniform(1e-4, 1),
    "batch_size": stats.randint(32, 512),
}

initial_config = {
    "learning_rate": 0.1,
    "batch_size": 256,
}

# Run multi-fidelity HPO (Successive Halving)
best_config, best_score, tuner = HPO.multi_fidelity_random_search(
    args, config_space=config_space, initial_config=initial_config
)

# Plot progress
HPO.plot_hpo_progress(tuner)

# Train final model with best config
print(f"\nTraining final model with best config: {best_config}")
train_loader, val_loader, test_loader = Utils.load_fashion_mnist(
    best_config["batch_size"]
)
model = Utils.build_model(args, best_config)
trainer = Trainer(
    model,
    train_loader,
    val_loader,
    test_loader,
    lr=best_config["learning_rate"],
    num_epochs=args.max_number_of_epochs,  # Use max epochs for final training
)
trainer.fit()

train_acc = trainer.evaluate_train()
val_acc = 1.0 - trainer.validation_error()
test_acc = trainer.evaluate_test()

print(f"Final train accuracy with Successive Halving-tuned hyperparameters: {train_acc:.4f}")
print(f"Final validation accuracy with Successive Halving-tuned hyperparameters: {val_acc:.4f}")
print(f"Final test accuracy with Successive Halving-tuned hyperparameters: {test_acc:.4f}")

## Example 3: Random Search - ASHA (LeNet)

ASHA is an asynchronous version of Successive Halving that promotes configs without waiting for synchronization barriers.

In [ ]:
# Configure arguments for ASHA
args = Args()
args.model_name = "lenet"
args.num_trials = 20
args.eta = 2
args.min_number_of_epochs = 5
args.max_number_of_epochs = 20
args.prefact = 1

# Define search space
config_space = {
    "learning_rate": stats.loguniform(1e-4, 1),
    "batch_size": stats.randint(32, 512),
}

initial_config = {
    "learning_rate": 0.1,
    "batch_size": 256,
}

# Run ASHA
best_config, best_score, tuner = HPO.asha_random_search(
    args, config_space=config_space, initial_config=initial_config
)

# Plot progress
HPO.plot_hpo_progress(tuner)

# Train final model with best config
print(f"\nTraining final model with best config: {best_config}")
train_loader, val_loader, test_loader = Utils.load_fashion_mnist(
    best_config["batch_size"]
)
model = Utils.build_model(args, best_config)
trainer = Trainer(
    model,
    train_loader,
    val_loader,
    test_loader,
    lr=best_config["learning_rate"],
    num_epochs=args.max_number_of_epochs,  # Use max epochs for final training
)
trainer.fit()

train_acc = trainer.evaluate_train()
val_acc = 1.0 - trainer.validation_error()
test_acc = trainer.evaluate_test()

print(f"Final train accuracy with ASHA-tuned hyperparameters: {train_acc:.4f}")
print(f"Final validation accuracy with ASHA-tuned hyperparameters: {val_acc:.4f}")
print(f"Final test accuracy with ASHA-tuned hyperparameters: {test_acc:.4f}")

## Bayesian Optimization with Model-Specific Hyperparameters

The following examples show how to optimize model-specific hyperparameters (alpha, l1_ratio, C, gamma) using Bayesian Optimization.

Bayesian Optimization uses a Gaussian Process to model the objective function and Expected Improvement to select promising configurations.

## Example 4: Bayesian Optimization (SVMClassifier)

Optimize SVM's penalty parameter `C` and RBF kernel coefficient `gamma`.

In [ ]:
# Configure for SVMClassifier
args = Args()
args.model_name = "svm"
args.num_trials = 15
args.num_epochs = 10

# Define search space for SVM (optimize C and gamma)
config_space = {
    "learning_rate": stats.loguniform(1e-4, 1),
    "batch_size": stats.randint(32, 512),
    "C": stats.loguniform(1e-2, 1e2),           # Penalty parameter
    "gamma": stats.loguniform(1e-6, 1e-1),      # RBF kernel coefficient
}

initial_config = {
    "learning_rate": 0.01,
    "batch_size": 256,
    "C": 1.0,
    "gamma": 0.001,
}

# Run Bayesian Optimization
best_config, best_score, tuner = HPO.bayesian_search(
    args, 
    config_space=config_space, 
    initial_config=initial_config,
    n_random_init=5
)

# Plot progress
HPO.plot_hpo_progress(tuner)

print(f"\n{'='*32}")
print("Best SVMClassifier Configuration:")
print(f"  Learning rate: {best_config['learning_rate']:.6f}")
print(f"  Batch size: {best_config['batch_size']}")
print(f"  C (penalty): {best_config['C']:.4f}")
print(f"  Gamma (kernel coef): {best_config['gamma']:.6f}")

# Train final model with best config
print(f"\nTraining final model with best config: {best_config}")
train_loader, val_loader, test_loader = Utils.load_fashion_mnist(
    best_config["batch_size"]
)
model = Utils.build_model(args, best_config)
trainer = Trainer(
    model,
    train_loader,
    val_loader,
    test_loader,
    lr=best_config["learning_rate"],
    num_epochs=args.num_epochs,
)
trainer.fit()

train_acc = trainer.evaluate_train()
val_acc = 1.0 - trainer.validation_error()
test_acc = trainer.evaluate_test()

print(f"Final train accuracy with Bayesian-tuned hyperparameters: {train_acc:.4f}")
print(f"Final validation accuracy with Bayesian-tuned hyperparameters: {val_acc:.4f}")
print(f"Final test accuracy with Bayesian-tuned hyperparameters: {test_acc:.4f}")

## Example 4: Bayesian Optimization (SGDClassifier)

Optimize SGD's regularization strength `alpha` and Elastic Net mixing parameter `l1_ratio`.

In [ ]:
# Configure for SGDClassifier
args = Args()
args.model_name = "sgd"
args.num_trials = 15
args.num_epochs = 10

# Define search space for SGD (optimize alpha and l1_ratio)
config_space = {
    "learning_rate": stats.loguniform(1e-4, 1),
    "batch_size": stats.randint(32, 512),
    "alpha": stats.loguniform(1e-6, 1e-1),      # Regularization strength
    "l1_ratio": stats.uniform(0, 1),            # Elastic net mixing (0=L2, 1=L1)
}

initial_config = {
    "learning_rate": 0.01,
    "batch_size": 256,
    "alpha": 0.0001,
    "l1_ratio": 0.15,
}

# Run Bayesian Optimization
best_config, best_score, tuner = HPO.bayesian_search(
    args, 
    config_space=config_space, 
    initial_config=initial_config,
    n_random_init=5
)

# Plot progress
HPO.plot_hpo_progress(tuner)

print(f"\n{'='*32}")
print("Best SGDClassifier Configuration:")
print(f"  Learning rate: {best_config['learning_rate']:.6f}")
print(f"  Batch size: {best_config['batch_size']}")
print(f"  Alpha (regularization): {best_config['alpha']:.6f}")
print(f"  L1 ratio: {best_config['l1_ratio']:.4f}")

# Train final model with best config
print(f"\nTraining final model with best config: {best_config}")
train_loader, val_loader, test_loader = Utils.load_fashion_mnist(
    best_config["batch_size"]
)
model = Utils.build_model(args, best_config)
trainer = Trainer(
    model,
    train_loader,
    val_loader,
    test_loader,
    lr=best_config["learning_rate"],
    num_epochs=args.num_epochs,
)
trainer.fit()

train_acc = trainer.evaluate_train()
val_acc = 1.0 - trainer.validation_error()
test_acc = trainer.evaluate_test()

print(f"Final train accuracy with Bayesian-tuned hyperparameters: {train_acc:.4f}")
print(f"Final validation accuracy with Bayesian-tuned hyperparameters: {val_acc:.4f}")
print(f"Final test accuracy with Bayesian-tuned hyperparameters: {test_acc:.4f}")

## Example 5: Bayesian Optimization (LeNet)

Apply Bayesian Optimization to LeNet (only optimizing learning_rate and batch_size).

In [ ]:
# Configure arguments for Bayesian Optimization
args = Args()
args.model_name = "lenet"
args.num_trials = 15  # 5 random init + 10 BO iterations
args.num_epochs = 10

# Define search space
config_space = {
    "learning_rate": stats.loguniform(1e-4, 1),
    "batch_size": stats.randint(32, 512),
}

initial_config = {
    "learning_rate": 0.1,
    "batch_size": 256,
}

# Run Bayesian Optimization
best_config, best_score, tuner = HPO.bayesian_search(
    args, 
    config_space=config_space, 
    initial_config=initial_config,
    n_random_init=5
)

# Plot progress
HPO.plot_hpo_progress(tuner)

# Train final model with best config
print(f"\nTraining final model with best config: {best_config}")
train_loader, val_loader, test_loader = Utils.load_fashion_mnist(
    best_config["batch_size"]
)
model = Utils.build_model(args, best_config)
trainer = Trainer(
    model,
    train_loader,
    val_loader,
    test_loader,
    lr=best_config["learning_rate"],
    num_epochs=args.num_epochs,
)
trainer.fit()

train_acc = trainer.evaluate_train()
val_acc = 1.0 - trainer.validation_error()
test_acc = trainer.evaluate_test()

print(f"Final train accuracy with Bayesian-tuned hyperparameters: {train_acc:.4f}")
print(f"Final validation accuracy with Bayesian-tuned hyperparameters: {val_acc:.4f}")
print(f"Final test accuracy with Bayesian-tuned hyperparameters: {test_acc:.4f}")

## Summary and Key Takeaways

### HPO Algorithms Comparison:

| Algorithm | Search Strategy | Multi-Fidelity | Model-Based | Synchronous | Best Use Case |
|-----------|----------------|----------------|-------------|-------------|---------------|
| **Random Search** | Random sampling | No | No | Yes | Baseline, small search spaces |
| **Bayesian Optimization** | GP + EI | No | Yes | Yes | Expensive evaluations, continuous spaces |
| **Successive Halving** | Random + early stopping | Yes | No | Yes | Limited parallel workers |
| **ASHA** | Random + early stopping | Yes | No | No | Many parallel workers, large-scale |

### Models Supported:

| Model | Type | Key Hyperparameters | Typical Accuracy | Speed |
|-------|------|---------------------|------------------|-------|
| **SoftmaxRegression** | Linear | lr, batch_size | ~82-85% | Fast |
| **LeNet** | CNN | lr, batch_size | ~85-90% | Slow |
| **SGDClassifier** | Linear SVM | lr, batch_size, alpha, l1_ratio | ~80-83% | Fast |
| **SVMClassifier** | Kernel SVM | lr, batch_size, C, gamma | ~83-87% | Moderate |

### Key Concepts Explained:

#### Multi-Fidelity Optimization:
- **Resource**: In our implementation, resource = `num_epochs` (training duration)
- **Rung**: A checkpoint where configs are evaluated with a specific resource level
- **eta (η)**: Reduction factor determining what fraction of configs advance (keep 1/η configs at each rung)
- **r_min**: Minimum resource (minimum number of epochs, e.g., 5)
- **r_max**: Maximum resource (maximum number of epochs, e.g., 20)
- **Rung Levels**: `[r_min, r_min*η, r_min*η², ..., r_max]`

Example with η=2, r_min=5, r_max=20:
- Rungs: [5, 10, 20] epochs
- If you start with 16 configs at rung 5, keep top 8 for rung 10, keep top 4 for rung 20

#### Bayesian Optimization Components:
- **Surrogate Model**: Gaussian Process with Matérn 5/2 kernel
  - Models the unknown objective function: f(x) ~ GP(μ, k)
  - Provides uncertainty estimates (mean and variance)
- **Acquisition Function**: Expected Improvement (EI)
  - EI(x) = E[max(f_min - f(x), 0)]
  - Balances exploitation (promising areas) vs exploration (uncertain areas)
  - Higher EI = more promising to evaluate
- **Optimization**: L-BFGS-B with 25 random restarts
  - Finds the x that maximizes EI(x)
  - Multiple restarts to avoid local optima
- **Random Initialization**: First n_random_init trials (default 5)
  - Seeds the GP with diverse observations
  - Ensures good coverage before starting BO

#### Config Space Distributions (scipy.stats):
- **`stats.loguniform(a, b)`**: Log-uniform distribution for learning rates, regularization parameters
  - Samples uniformly in log-space: log(x) ~ Uniform(log(a), log(b))
  - Good for parameters spanning multiple orders of magnitude
  - Attributes: `domain.a` (lower bound), `domain.b` (upper bound)
- **`stats.randint(low, high)`**: Discrete uniform for batch sizes
  - Samples integers uniformly: x ~ Uniform({low, low+1, ..., high-1})
- **`stats.uniform(a, b)`**: Uniform distribution for l1_ratio
  - Samples uniformly: x ~ Uniform(a, b)

**Important**: Use `scipy.stats` distributions (not `syne_tune`) for Bayesian Optimization because:
- BayesianSearcher accesses `domain.a` and `domain.b` attributes
- These attributes are specific to scipy.stats frozen distributions
- Enables proper bounds for L-BFGS-B optimization

### ASHA vs Successive Halving:

**Successive Halving (Synchronous):**
- Waits for all configs to complete at a rung before promoting
- Synchronization barriers can cause worker idle time
- All promotions happen together in batches

**ASHA (Asynchronous):**
- Promotes configs as soon as enough results are available
- No synchronization barriers
- Workers never idle waiting for others
- Better resource utilization in distributed settings
- Checks rungs from top to bottom for promotion opportunities

### Implementation Details:

#### 1. Model Building with Config:
```python
# Utils.build_model(args, config) handles model-specific hyperparameters
model = Utils.build_model(args, config)

# For SGD: extracts alpha and l1_ratio from config
# For SVM: extracts C and gamma from config
# Uses args.alpha, args.C, etc. as defaults if not in config
```

#### 2. HPO Objective Function:
```python
def hpo_objective(config):
    # Extract hyperparameters
    lr = config.get("learning_rate", args.learning_rate)
    batch_size = config.get("batch_size", args.batch_size)
    num_epochs = config.get("num_epochs", args.num_epochs)
    
    # Build model (automatically handles model-specific params)
    model = Utils.build_model(args, config)
    
    # Train and evaluate
    trainer = Trainer(model, ..., lr=lr, num_epochs=num_epochs)
    trainer.fit()
    return trainer.validation_error()
```

#### 3. Final Training:
After HPO finds the best config, train final model:
```python
train_loader, val_loader, test_loader = Utils.load_fashion_mnist(best_config["batch_size"])
model = Utils.build_model(args, best_config)  # Automatically sets alpha, C, gamma, etc.
trainer = Trainer(model, ..., lr=best_config["learning_rate"], num_epochs=args.num_epochs)
trainer.fit()
```

### Recommended Settings:

**Random Search:**
- `num_trials`: 10-20 trials
- Good baseline for comparison
- Works well when you have prior knowledge of good regions

**Bayesian Optimization:**
- `num_trials`: 15-30 (5 random init + 10-25 BO iterations)
- `n_random_init`: 5 (rule of thumb: 1-2 per hyperparameter)
- Best for: 2-6 continuous hyperparameters, expensive models (LeNet, SVM)
- Advantage: More sample-efficient than random search

**Successive Halving:**
- `num_trials`: 20-50
- `eta`: 2 or 3 (2 = aggressive pruning, 3 = more conservative)
- `r_min`, `r_max`: Based on your model (e.g., 5-20 epochs for FashionMNIST)
- Best for: Limited parallel workers (1-4 workers)

**ASHA:**
- `num_trials`: 50-100+ (scales well with more trials)
- `eta`: 2 or 3
- `r_min`, `r_max`: Same as Successive Halving
- Best for: Many parallel workers (4+ workers), large-scale experiments
- Advantage: No worker idle time, scales efficiently

### Model-Specific Hyperparameter Ranges:

**SGDClassifier:**
- `alpha`: `stats.loguniform(1e-6, 1e-1)` - Regularization strength
  - Lower = less regularization (may overfit)
  - Higher = more regularization (may underfit)
- `l1_ratio`: `stats.uniform(0, 1)` - Elastic net mixing
  - 0.0 = pure L2 (Ridge regression)
  - 1.0 = pure L1 (Lasso, promotes sparsity)
  - 0.15 = balanced (good default)

**SVMClassifier:**
- `C`: `stats.loguniform(1e-2, 1e2)` - Penalty parameter
  - Lower = more regularization (simpler model)
  - Higher = less regularization (more complex decision boundary)
- `gamma`: `stats.loguniform(1e-6, 1e-1)` - RBF kernel coefficient
  - Lower = smoother decision boundary (more global)
  - Higher = tighter decision boundary (more local)

### Performance Tips:

1. **Model Selection:**
   - LeNet: Best accuracy (~85-90%) but slowest to train
   - SoftmaxRegression: Fast baseline (~82-85%)
   - SGDClassifier: Fast, good for linear problems (~80-83%)
   - SVMClassifier: Good non-linear performance (~83-87%), moderate speed

2. **Algorithm Selection:**
   - Use **Bayesian BO** for expensive models (LeNet, SVM) with few hyperparameters
   - Use **Random Search** as baseline or when search space is simple
   - Use **ASHA** when you have many parallel workers
   - Use **Successive Halving** when workers are limited

3. **Hyperparameter Search Spaces:**
   - Always use log-scale for learning rates: `stats.loguniform(1e-4, 1)`
   - Always use log-scale for regularization: `stats.loguniform(1e-6, 1e-1)`
   - Use linear scale for mixing parameters: `stats.uniform(0, 1)`
   - Batch size can be `randint(32, 512)` or powers of 2

4. **Multi-Fidelity Settings:**
   - Choose r_min to be small enough to quickly identify bad configs (e.g., 5 epochs)
   - Choose r_max based on when model converges (e.g., 20 epochs for FashionMNIST)
   - η=2 is aggressive (halves configs at each rung), η=3 is more conservative

### Next Steps:

1. **Compare Algorithms**: Run all 4 methods on same model and compare:
   - Sample efficiency (which finds best config with fewer trials?)
   - Time efficiency (which finds good config fastest?)
   - Final performance (which achieves best validation accuracy?)

2. **Try Different Models**: Test SGD/SVM vs LeNet on FashionMNIST
   - How much accuracy do you sacrifice for speed?
   - Which hyperparameters matter most?

3. **Scale Up**: Apply to larger datasets (CIFAR-10, ImageNet)
   - Use ASHA with many parallel workers
   - Increase r_max for more complex datasets

4. **Combine Methods**: 
   - BOHB (Bayesian Optimization + Hyperband) = BO with multi-fidelity
   - Use BO as searcher instead of RandomSearcher in ASHA

5. **Add Constraints**:
   - Constrained optimization (e.g., accuracy ≥ 85% AND latency ≤ 100ms)
   - Multi-objective optimization (accuracy vs inference speed)

6. **Advanced Topics**:
   - Transfer learning: Use knowledge from previous HPO runs
   - Meta-learning: Learn which configs work well across tasks
   - Neural Architecture Search (NAS): Optimize model architecture itself

### Common Pitfalls:

1. **Using num_epochs with single-fidelity methods:**
   - Random Search and Bayesian BO don't use min/max_epochs
   - Only multi-fidelity methods (SH, ASHA) use varying epochs

2. **Forgetting model-specific hyperparameters:**
   - Must include alpha, l1_ratio in config_space for SGD
   - Must include C, gamma in config_space for SVM
   - Utils.build_model() handles extraction automatically

3. **Wrong distribution types:**
   - Use `stats.loguniform` for scipy-based BO (not `syne_tune.loguniform`)
   - Use `stats.randint` for discrete parameters (not `stats.uniform`)

4. **Insufficient random initialization:**
   - GP needs diverse observations to build good model
   - Use n_random_init ≥ 5 (or 1-2 per hyperparameter)

5. **Not training final model properly:**
   - For multi-fidelity: use `max_number_of_epochs` for final training
   - For single-fidelity: use `args.num_epochs` for final training
   - Always pass `best_config` to `Utils.build_model()`

### References:

- **Bayesian Optimization**: Snoek et al., "Practical Bayesian Optimization of Machine Learning Algorithms" (2012)
- **Successive Halving**: Jamieson & Talwalkar, "Non-stochastic Best Arm Identification and Hyperparameter Optimization" (2016)
- **ASHA**: Li et al., "A System for Massively Parallel Hyperparameter Tuning" (2018)
- **Random Fourier Features**: Rahimi & Recht, "Random Features for Large-Scale Kernel Machines" (2007)